<a href="https://colab.research.google.com/github/dsr-lab/nlp_assignments/blob/main/NLP_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Edit from Main 1
# Test
# Edit from Main 2